[주제] 테크 뉴스레터 작성하기

# CrewAI와 Tool 결합

In [1]:
import warnings
warnings.filterwarnings('ignore')

- Serper : Google Search API 키 발급
    - https://serper.dev/

In [3]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY= os.getenv('OPENAI_API_KEY')
SERPER_API_KEY= os.getenv('SERPER_API_KEY')

# 라이브러리 임포트
### crewai_tools
- SerperDevTool : Serper api를 활용한 결과물 받는 도구, 구글 웹검색 도구
- WebsiteSearchTool : 특정 키워드와 특정 웹사이트를 주면, 해당 웹 사이트에서 키워드와 관련이 있는 부부만 가져옴.
- ScrapeWebsiteTool : 어떤 URL을 주면 해당 url에 있는 텍스트를 스크래핑해서 가져오는 도구

In [4]:
from crewai import Agent, Task, Crew
from crewai.process import Process
from crewai_tools import SerperDevTool, WebsiteSearchTool, ScrapeWebsiteTool
from langchain_openai import ChatOpenAI

## 필요한 객체 생성

In [5]:
# llm 객체
llm = ChatOpenAI(model="gpt-4o-mini")

# Tool
search_tool = SerperDevTool()
web_rag_tool = WebsiteSearchTool()
scrap_tool = ScrapeWebsiteTool()

## Agent 정의

In [6]:
# 테크 트렌드 연구원
researcher=Agent(
    name="tech_trend_researcher",
    role='테크 트렌드 연구원',
    goal='인공지능 분야의 최신 기술 트렌드를 한국어로 제공합니다. 지금은 2026년 1월입니다.',
    backstory='기술 트렌드에 예리한 안목을 지닌 전문 분석가이자 AI 개발자입니다.',
    tools=[search_tool, web_rag_tool],
    verbose=True,
    mac_iter=5, # 해당 Agent가 주어진 task를 수행할 때 반복 실행할 횟수를 제한하는 것임(웹검색을 5번 실행, 비용과 연결되어 있음.)
    llm=llm
)

- allow_delegation=False
    - 해당 agnet가 task를 수행할 때 혼자 힘으로 하기 힘들 때 다른 agnet에 위임하는 설정
    - detault=True
    - False로 하지 않으면 수행중에 researcher에게 작업을 넘기는 경우가 발생하는데 이것을 발생하지 않도록 제한하기 위함

In [7]:
# 뉴스레터 작성 agent
writer = Agent(
    name="tech_newsletter_writer",
    role="뉴스레터 작성자",
    goal="최신 AI 기술 트렌드에 대한 매력적인 테크 뉴스레터를 한국어로 작성하세요. 지금은 2026년 1월입니다.",
    backstory="기술에 대한 열정을 가진 숙련된 작가입니다.",
    verbose=True,
    allow_delegation=False,  
    llm=llm
)

## Task 정의

In [8]:
research = Task(
    name="tech_trend_research",
    description="AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.",
    expected_output="AI 업계에서 가장 주목받는 3대 기술 동향과 그 중요성에 대한 신선한 관점을 요약한 글",
    agent=researcher
)

In [9]:
write = Task(
    name="tech_newsletter_writing",
    description="""
테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.
테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.
""",
    expected_output="최신 기술 관련 소식을 재밌는 말투로 소개하는 4문단짜리 마크다운 형식 뉴스레터", # 결과물 형식
    agent=writer,
    output_file=r"./new_post.md" # the final blog post will be saved here
)

## Crew 정의

In [10]:
crew = Crew(
    agents=[researcher, writer],
    tasks=[research, write],
    verbose=False,
    process=Process.sequential,
    output_log_file="logs/crew_run.json",   # 👈 JSON 형태로 모든 스텝이 저장됩니다.
)

## Crew 실행

In [12]:
import json

import os

os.makedirs("logs", exist_ok=True)

result = crew.kickoff()

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 테크 트렌드 연구원                                                                                      │
│                                                                                                                 │
│  Task: AI 업계의 최신 기술 동향을 조사하고 요약을 제공하세요.                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {'searchParameters': {'q': '2026 AI trends', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic':      │
│  [{'title': 'The trends that will shape AI and tech in 2026', 'link':                                           │
│  'https://www.ibm.com/think/news/ai-tech-trends-predictions-2026', 'snippet': 'Reporter Anabelle Nicoud spoke   │
│  to several experts across AI, security, quantum and beyond to better understand where tech will take us in     │
│  2026.', 'position': 1}, {'title': 'Five Trends in AI and Data Science for 2026', 'link':                       │
│  'https://sloanreview.mit.edu/article/five-trends-in-ai-and-data-science-for-2026/', 'snippet': 'Summary: MIT   │
│  SMR columnists Thomas H. Davenport and Randy Bean see five AI trends to pay attention to in 2026: deflation    │
│  of the AI ... ...', 'position': 2}, {'title': "What's next in AI: 7 trends to watch in 2026", 'link':          │
│  'https://news.microsoft.com/source/features/ai/whats-next-in-ai-7-trends-to-watch-in-2026/', 'snippet':        │
│  'Seven AI trends to watch in 2026 will make AI a true partner — boosting teamwork, security, research          │
│  momentum and infrastructure efficiency.', 'position': 3}, {'title': 'Top 6 AI Trends That Will Define 2026     │
│  (backed by data)', 'link': 'https://www.youtube.com/watch?v=B23W1gRT9eY', 'snippet': 'Here are the six ai      │
│  trends that will matter most in 2026. This list is based on daily research and reports from institutions like  │
│  mckenzie openai stanford.', 'position': 4}, {'title': "What's next for AI in 2026 | MIT Technology ...",       │
│  'link': 'https://www.technologyreview.com/2026/01/05/1130662/whats-next-for-ai-in-2026/', 'snippet': 'In       │
│  2026, expect more Silicon Valley apps to quietly ship on top of Chinese open models, and look for the lag      │
│  between Chinese releases and the ...', 'position': 5}, {'title': 'AI agent trends 2026 report', 'link':        │
│  'https://cloud.google.com/resources/content/ai-agent-trends-2026', 'snippet': 'Our new report reveals the 5    │
│  top trends in agentic AI that can help transform businesses, for 2026 and beyond. Download your copy.',        │
│  'position': 6}, {'title': 'Tech Trend...                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│  No relevant content found.                                                                                     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│  The trends that will shape AI and tech in 2026 | IBM                                                           │
│                                                                                                                 │
│  Artificial Intelligence                                                                                        │
│                                                                                                                 │
│  Statements and reactions                                                                                       │
│                                                                                                                 │
│  The trends that will shape AI and tech in 2026                                                                 │
│                                                                                                                 │
│  Published 01 January 2026                                                                                      │
│                                                                                                                 │
│  Anabelle Nicoud                                                                                                │
│                                                                                                                 │
│  Staff Writer                                                                                                   │
│                                                                                                                 │
│  IBM                                                                                                            │
│                                                                                                                 │
│  This article was featured in the Think newsletter. Get it in your inbox .                                      │
│                                                                                                                 │
│  A year in tech can feel like a decade anywhere else.                                                           │
│                                                                                                                 │
│  Think about it: a year ago, we were discussing how ChatGPT wasn’t able to count the number of “r”s in          │
│  “strawberry.” Reasoning models from Chinese frontier labs (like DeepSeek-R1 ) hadn’t taken the world by        │
│  storm, and neither had open-source reasoning agents.                                                           │
│                                                                                                                 │
│  Claude’s dedicated coding agent didn’t exist yet. IBM’s Granite 3.0 had only just arrived . And the agent      │
│  conversation was only beginning: MCP had just gained traction in the spring, with a notable endorsement from   │
│  Sam Altman.                                                                                                    │
│                                                                                                                 │
│  Meanwhile, in the world of infrastructure, chips and compute resources were becoming scarce , giving new       │
│  territories a competitive advantage.                                                                           │
│                                                                                                                 │
│  Over the last few weeks, IBM Think spoke with a dozen

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Relevant Content:                                                                                              │
│  The trends that will shape AI and tech in 2026 | IBM                                                           │
│                                                                                                                 │
│  Artificial Intelligence                                                                                        │
│                                                                                                                 │
│  Statements and reactions                                                                                       │
│                                                                                                                 │
│  The trends that will shape AI and tech in 2026                                                                 │
│                                                                                                                 │
│  Published 01 January 2026                                                                                      │
│                                                                                                                 │
│  Anabelle Nicoud                                                                                                │
│                                                                                                                 │
│  Staff Writer                                                                                                   │
│                                                                                                                 │
│  IBM                                                                                                            │
│                                                                                                                 │
│  This article was featured in the Think newsletter. Get it in your inbox .                                      │
│                                                                                                                 │
│  A year in tech can feel like a decade anywhere else.                                                           │
│                                                                                                                 │
│  Think about it: a year ago, we were discussing how ChatGPT wasn’t able to count the number of “r”s in          │
│  “strawberry.” Reasoning models from Chinese frontier labs (like DeepSeek-R1 ) hadn’t taken the world by        │
│  storm, and neither had open-source reasoning agents.                                                           │
│                                                                                                                 │
│  Claude’s dedicated coding agent didn’t exist yet. IBM’s Granite 3.0 had only just arrived . And the agent      │
│  conversation was only beginning: MCP had just gained traction in the spring, with a notable endorsement from   │
│  Sam Altman.                                                                                                    │
│                                                                                                                 │
│  Meanwhile, in the world of infrastructure, chips and compute resources were becoming scarce , giving new       │
│  territories a competitive advantage.                                                                           │
│                                                                                                                 │
│  Over the last few weeks, IBM Think spoke with a dozen

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 테크 트렌드 연구원                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  2026년 AI 업계의 최신 기술 동향은 다음과 같습니다:                                                             │
│                                                                                                                 │
│  1. **AI의 자동화 및 업무 복잡성 해결**                                                                         │
│     AI는 이제 단순한 질문에 답하는 것을 넘어서, 기업의 복잡한 업무 흐름을 자동화하는 데 중점을 두고 있습니다.   │
│  예를 들어, AI는 사용자의 의도를 해석하고, 다양한 네트워크를 통해 필요한 도구를 선택하여 목표에 도달할 때까지   │
│  작업을 수행할 수 있습니다. 이는 업무의 효율성을 극대화하며, AI의 역할을 우리가 사용하고 의지하는 단순한        │
│  도구에서 적극적인 '비즈니스 파트너'로 변화시킵니다. 특히 로보틱스와 물리적 AI의 발전은 환경에서 직접 작동하고  │
│  학습할 수 있는 AI의 필요성을 증가시키고 있습니다.                                                              │
│                                                                                                                 │
│  2. **오픈소스의 중요성 증가**                                                                                  │
│     오픈소스 AI는 세계적으로 모델의 다양화와 상호운용성을 통해 더욱 발전하고 있습니다. 이는 작은 모델들이 특정  │
│  도메인에 최적화되어 산업 전반에서 사용될 수 있음을 시사합니다. 오픈소스 AI의 발전은 기업들이 GPU 기반의 협력   │
│  생태계에서 경쟁력을 갖추도록 도와줍니다. 특히, 많은 기업들이 AI를 이용하여 사업 모델을 혁신하고 신뢰성과       │
│  투명성을 강화하는 데 집중하고 있습니다.                                                                        │
│                                                                                                                 │
│  3. **AI 주권 및 데이터 보안의 중요성**                                                                         │
│     AI 시스템의 정부 및 데이터 관리 능력, 즉 AI 주권은 기업의 전략에서 핵심 요소로 떠오르고 있습니다. 많은      │
│  경영진은 이 비용에 대해 우려하고 있으며, 모든 AI 에이전트의 액세스 및 활동을 실시간으로 파악하고 관리하는      │
│  것이 기업의 보안 전략에 중요한 부분으로 자리잡고 있습니다. 데이터의 보안과 트러스트는 머신러닝 모델의 성과를   │
│  유지하기 위해 필수적입니다.                                                                                    │
│                                                                                                                 │
│  이 세 가지 동향은 AI 기술이 기업의 운영 및 의사결정 방식에 어떻게 큰 영향을 미칠 것인지 보여줍니다. 2026년의   │
│  AI 업계는 이러한 방향으로 급격하게 변화하고 있음을 알 수 있습니다.                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 뉴스레터 작성자                                                                                         │
│                                                                                                                 │
│  Task:                                                                                                          │
│  테크 트렌드 연구원의 요약을 바탕으로 AI 산업에 대한 매력적인 테크 뉴스레터를 작성하세요.                       │
│  테크 뉴스레터이므로 전문적인 용어를 사용해도 괜찮습니다.                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: 뉴스레터 작성자                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # 📰 AI 뉴스레터 - 2026년 1월호                                                                                │
│                                                                                                                 │
│  안녕하세요, AI 애호가 여러분! 2026년의 첫 달이 밝았습니다. 오늘은 최신 AI 산업 트렌드에 대한 흥미로운          │
│  소식들을 가져왔습니다. AI는 단순한 도구에서 적극적인 비즈니스 파트너로 떠오르고 있는지, 한번 살펴볼까요?       │
│                                                                                                                 │
│  ## 🤖 AI의 자동화: 복잡함을 간단하게!                                                                          │
│  AI는 이제 더 이상 단순한 질문에 답하는 걸 넘어, 복잡한 기업 업무를 자동화하는 데 집중하고 있습니다. 최신 AI    │
│  시스템은 사용자의 의도를 해석하여 다양한 도구를 선택하고, 최종 목표에 도달할 수 있도록 지속적으로 학습합니다.  │
│  로보틱스와 물리적 AI가 진화하면서, 다양한 환경에서 작동하고 스스로 습득하는 능력이 증대되고 있습니다. 이는     │
│  AI가 단순한 보조 도구가 아닌, 진정한 비즈니스 파트너로 발전하는 계기를 마련하고 있습니다!                      │
│                                                                                                                 │
│  ## 🌐 오픈소스의 혁신: 함께 만드는 AI                                                                          │
│  오픈소스 AI의 세계가 더욱 다양해지고 있습니다! 여러 기업들이 GPU 기반의 협력 생태계를 통해 효율적인 AI 모델을  │
│  개발하고 있는 가운데, 이들 작은 모델들은 특정 도메인에 최적화되어 산업 전반에서 경쟁력을 높이고 있습니다.      │
│  기업들이 신뢰성과 투명성을 강화하며 사업 모델 혁신에 힘쓰는 모습이 인상적입니다. AI 시대의 협력 정신이 더욱    │
│  중요해지는 만큼, 오픈소스의 발전을 주목해야겠죠!                                                               │
│                                                                                                                 │
│  ## 🔐 AI 주권: 보안의 새로운 패러다임                                                                          │
│  AI 주권과 데이터 보안은 이제 기업 전략의 핵심 요소로 자리잡고 있습니다. 많은 경영진들이 AI 시스템의 관리       │
│  능력에 대해 우려하고 있으며, 실시간으로 모든 AI 에이전트의 액세스 및 활동을 모니터링하는 것이 보안 전략의      │
│  중요한 부분으로 자리 잡고 있습니다. 데이터 보안과 신뢰성은 AI의 성공적인 운영에 필수적이며, 이를 관리하는      │
│  기업들이 앞서나가겠죠!                                                                                         │
│                                                                                                                 │
│  2026년이 시작되면서 AI 기술이 기업의 운영 및 의사결정 방식에 미치는 영향은 날로 커지고 있습니다. 이러한        │
│  흥미로운 변화들을 계속 지켜보며, 여러분의 비즈니스에 도움이 되길 바라며 이번 뉴스레터를 마칩니다. 다음         │
│  호에서도 AI의 새로운 소식을 만나보세요! 🌟                                                                     │
│  ```                                                                                                            │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## 마크다운으로 표시

In [13]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

```markdown
# 📰 AI 뉴스레터 - 2026년 1월호

안녕하세요, AI 애호가 여러분! 2026년의 첫 달이 밝았습니다. 오늘은 최신 AI 산업 트렌드에 대한 흥미로운 소식들을 가져왔습니다. AI는 단순한 도구에서 적극적인 비즈니스 파트너로 떠오르고 있는지, 한번 살펴볼까요?

## 🤖 AI의 자동화: 복잡함을 간단하게!
AI는 이제 더 이상 단순한 질문에 답하는 걸 넘어, 복잡한 기업 업무를 자동화하는 데 집중하고 있습니다. 최신 AI 시스템은 사용자의 의도를 해석하여 다양한 도구를 선택하고, 최종 목표에 도달할 수 있도록 지속적으로 학습합니다. 로보틱스와 물리적 AI가 진화하면서, 다양한 환경에서 작동하고 스스로 습득하는 능력이 증대되고 있습니다. 이는 AI가 단순한 보조 도구가 아닌, 진정한 비즈니스 파트너로 발전하는 계기를 마련하고 있습니다!

## 🌐 오픈소스의 혁신: 함께 만드는 AI
오픈소스 AI의 세계가 더욱 다양해지고 있습니다! 여러 기업들이 GPU 기반의 협력 생태계를 통해 효율적인 AI 모델을 개발하고 있는 가운데, 이들 작은 모델들은 특정 도메인에 최적화되어 산업 전반에서 경쟁력을 높이고 있습니다. 기업들이 신뢰성과 투명성을 강화하며 사업 모델 혁신에 힘쓰는 모습이 인상적입니다. AI 시대의 협력 정신이 더욱 중요해지는 만큼, 오픈소스의 발전을 주목해야겠죠!

## 🔐 AI 주권: 보안의 새로운 패러다임
AI 주권과 데이터 보안은 이제 기업 전략의 핵심 요소로 자리잡고 있습니다. 많은 경영진들이 AI 시스템의 관리 능력에 대해 우려하고 있으며, 실시간으로 모든 AI 에이전트의 액세스 및 활동을 모니터링하는 것이 보안 전략의 중요한 부분으로 자리 잡고 있습니다. 데이터 보안과 신뢰성은 AI의 성공적인 운영에 필수적이며, 이를 관리하는 기업들이 앞서나가겠죠!

2026년이 시작되면서 AI 기술이 기업의 운영 및 의사결정 방식에 미치는 영향은 날로 커지고 있습니다. 이러한 흥미로운 변화들을 계속 지켜보며, 여러분의 비즈니스에 도움이 되길 바라며 이번 뉴스레터를 마칩니다. 다음 호에서도 AI의 새로운 소식을 만나보세요! 🌟
```

[tip] log 후처리
- logs/crew_run.json의 로그 결과에서 한글을 하기 위한 JSON 변환하기

In [14]:
import json

with open("logs/crew_run.json", 'r', encoding="utf-8") as f:
    data=json.load(f)

with open("logs/crew_run_readable.jsons", 'w', encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

    print("crew_run_readable.json 생성 완료")

crew_run_readable.json 생성 완료
